In [75]:
#import dependencies
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import LSTM, Dense, InputLayer, Dropout, BatchNormalization
from keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score
from keras.regularizers import l2

In [76]:
#create df of preprocessed data - game data from 2004-2024
df = pd.read_csv('nbaPreProcessed.csv')
df

,date,homeTeam,homeTeam_id,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,...,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF,winner,season,isPlayoffGame,winner_binary
0,2024-06-17,Boston Celtics,BOS,2,0.792079,80,21,106,28,39,...,35,18,4,4,13,20,Boston Celtics,2023-2024,True,0
1,2024-06-14,Dallas Mavericks,DAL,7,0.611650,63,40,122,34,27,...,31,18,2,5,13,19,Dallas Mavericks,2023-2024,True,0
2,2024-06-12,Dallas Mavericks,DAL,7,0.607843,62,40,99,31,20,...,36,26,4,6,9,19,Boston Celtics,2023-2024,True,1
3,2024-06-09,Boston Celtics,BOS,2,0.795918,78,20,105,25,29,...,43,21,5,3,15,17,Boston Celtics,2023-2024,True,0
4,2024-06-06,Boston Celtics,BOS,2,0.793814,77,20,107,37,26,...,43,9,8,1,11,16,Boston Celtics,2023-2024,True,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,Cleveland Cavaliers,CLE,6,0.000000,0,1,104,21,19,...,47,24,9,5,13,28,Indiana Pacers,2004-2005,False,1
25727,2004-11-03,Boston Celtics,BOS,2,0.000000,0,1,95,22,31,...,41,20,9,2,16,26,Philadelphia 76ers,2004-2005,False,1
25728,2004-11-02,Los Angeles Lakers,LAL,14,1.000000,1,0,89,24,26,...,48,17,10,8,16,21,Los Angeles Lakers,2004-2005,False,0
25729,2004-11-02,Detroit Pistons,DET,9,1.000000,1,0,87,19,18,...,36,8,4,5,16,24,Detroit Pistons,2004-2005,False,0


In [77]:
#populate dfIDtoStat - should be double the size of df
homedf = df[['date', 'homeTeamSubject_id', 'homeTeam_win%', 'homeTeam_wins', 'homeTeam_losses', 'homeTeam_points_q1', 'homeTeam_points_q2', 'homeTeam_points_q3', 'homeTeam_points_q4', 'homeTeam_points_total', 'homeTeam_FG', 'homeTeam_FG_made', 'homeTeam_3P', 'homeTeam_3P_made', 'homeTeam_O_Reb', 'homeTeam_D_Reb', 'homeTeam_Total_Reb', 'homeTeam_Ast', 'homeTeam_Stl', 'homeTeam_Blk', 'homeTeam_TO', 'homeTeam_PF']]
homedf.columns = ['date', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_total', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']
awaydf = df[['date', 'awayTeamSubject_id', 'awayTeam_win%', 'awayTeam_wins', 'awayTeam_losses', 'awayTeam_points_q1', 'awayTeam_points_q2', 'awayTeam_points_q3', 'awayTeam_points_q4', 'awayTeam_points_total', 'awayTeam_FG', 'awayTeam_FG_made', 'awayTeam_3P', 'awayTeam_3P_made', 'awayTeam_O_Reb', 'awayTeam_D_Reb', 'awayTeam_Total_Reb', 'awayTeam_Ast', 'awayTeam_Stl', 'awayTeam_Blk', 'awayTeam_TO', 'awayTeam_PF']]
awaydf.columns = ['date', 'team_id', 'team_win%', 'team_wins', 'team_losses', 'team_points_q1', 'team_points_q2', 'team_points_q3', 'team_points_q4', 'team_points_total', 'team_FG', 'team_FG_made', 'team_3P', 'team_3P_made', 'team_O_Reb', 'team_D_Reb', 'team_Total_Reb', 'team_Ast', 'team_Stl', 'team_Blk', 'team_TO', 'team_PF']

dfIDtoStat = pd.concat([homedf, awaydf], ignore_index=True)
dfIDtoStat

,date,team_id,team_win%,team_wins,team_losses,team_points_q1,team_points_q2,team_points_q3,team_points_q4,team_points_total,...,team_3P,team_3P_made,team_O_Reb,team_D_Reb,team_Total_Reb,team_Ast,team_Stl,team_Blk,team_TO,team_PF
0,2024-06-17,2,0.792079,80,21,28,39,19,20,106,...,0.333333,13,15,36,51,25,9,2,7,15
1,2024-06-14,7,0.611650,63,40,34,27,31,30,122,...,0.405405,15,13,39,52,21,7,2,8,17
2,2024-06-12,7,0.607843,62,40,31,20,19,29,99,...,0.360000,9,7,36,43,15,5,1,8,17
3,2024-06-09,2,0.795918,78,20,25,29,29,22,105,...,0.256410,10,10,31,41,29,10,5,10,15
4,2024-06-06,2,0.793814,77,20,37,26,23,21,107,...,0.380952,16,10,37,47,23,6,9,12,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51457,2004-11-03,12,1.000000,1,0,26,20,17,22,109,...,0.235294,4,11,36,47,24,9,5,13,28
51458,2004-11-03,23,1.000000,1,0,27,24,20,27,98,...,0.375000,9,14,27,41,20,9,2,16,26
51459,2004-11-02,8,0.000000,0,1,14,17,26,21,78,...,0.250000,3,16,32,48,17,10,8,16,21
51460,2004-11-02,11,0.000000,0,1,18,20,21,20,79,...,0.461538,6,9,27,36,8,4,5,16,24


In [78]:
dfHome = pd.read_csv('home_performances.csv')
dfHome = dfHome.drop(['homeTeam', 'homeTeam_id', 'winner', 'season', 'isPlayoffGame', 'winner_binary'], axis=1)
dfHome

,date,homeTeamSubject_id,homeTeam_win%,homeTeam_wins,homeTeam_losses,homeTeam_points_total,homeTeam_points_q1,homeTeam_points_q2,homeTeam_points_q3,homeTeam_points_q4,...,homeTeam_3P,homeTeam_3P_made,homeTeam_O_Reb,homeTeam_D_Reb,homeTeam_Total_Reb,homeTeam_Ast,homeTeam_Stl,homeTeam_Blk,homeTeam_TO,homeTeam_PF
0,2024-06-17,2,0.792079,80,21,106,28,39,19,20,...,0.333333,13,15,36,51,25,9,2,7,15
1,2024-06-14,7,0.611650,63,40,122,34,27,31,30,...,0.405405,15,13,39,52,21,7,2,8,17
2,2024-06-12,7,0.607843,62,40,99,31,20,19,29,...,0.360000,9,7,36,43,15,5,1,8,17
3,2024-06-09,2,0.795918,78,20,105,25,29,29,22,...,0.256410,10,10,31,41,29,10,5,10,15
4,2024-06-06,2,0.793814,77,20,107,37,26,23,21,...,0.380952,16,10,37,47,23,6,9,12,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,6,0.000000,0,1,104,21,19,27,18,...,0.375000,3,11,34,45,23,3,8,14,24
25727,2004-11-03,2,0.000000,0,1,95,22,31,30,12,...,0.294118,5,10,25,35,23,9,3,19,27
25728,2004-11-02,14,1.000000,1,0,89,24,26,22,17,...,0.294118,5,10,37,47,23,8,9,19,22
25729,2004-11-02,9,1.000000,1,0,87,19,18,22,28,...,0.333333,6,13,29,42,20,9,5,12,22


In [79]:
dfAway = pd.read_csv('away_performances.csv')
dfAway = dfAway.drop(['awayTeam', 'awayTeam_id', 'winner', 'season', 'isPlayoffGame', 'winner_binary'], axis=1)
dfAway

,date,awayTeamSubject_id,awayTeam_win%,awayTeam_wins,awayTeam_losses,awayTeam_points_total,awayTeam_points_q1,awayTeam_points_q2,awayTeam_points_q3,awayTeam_points_q4,...,awayTeam_3P,awayTeam_3P_made,awayTeam_O_Reb,awayTeam_D_Reb,awayTeam_Total_Reb,awayTeam_Ast,awayTeam_Stl,awayTeam_Blk,awayTeam_TO,awayTeam_PF
0,2024-06-17,7,0.605769,63,41,88,18,28,21,21,...,0.297297,11,7,28,35,18,4,4,13,20
1,2024-06-14,2,0.790000,79,21,84,21,14,25,24,...,0.341463,14,4,27,31,18,2,5,13,19
2,2024-06-12,2,0.797980,79,20,106,30,20,35,21,...,0.369565,17,6,30,36,26,4,6,9,19
3,2024-06-09,7,0.613861,62,39,98,28,23,23,24,...,0.230769,6,9,34,43,21,5,3,15,17
4,2024-06-06,7,0.620000,62,38,89,20,22,24,23,...,0.259259,7,10,33,43,9,8,1,11,16
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25726,2004-11-03,12,1.000000,1,0,109,26,20,17,22,...,0.235294,4,11,36,47,24,9,5,13,28
25727,2004-11-03,23,1.000000,1,0,98,27,24,20,27,...,0.375000,9,14,27,41,20,9,2,16,26
25728,2004-11-02,8,0.000000,0,1,78,14,17,26,21,...,0.250000,3,16,32,48,17,10,8,16,21
25729,2004-11-02,11,0.000000,0,1,79,18,20,21,20,...,0.461538,6,9,27,36,8,4,5,16,24


In [80]:
windowConditions = {
    'team_win%': 1,
    'team_wins': 1,
    'team_losses': 1,
    'team_points_total': 17,
    'team_points_q1': 17,
    'team_points_q2': 17,
    'team_points_q3': 17,
    'team_points_q4': 20,
    'team_FG': 19,
    'team_FG_made': 17,
    'team_3P': 20,
    'team_3P_made': 15,
    'team_O_Reb': 17,
    'team_D_Reb': 15,
    'team_Total_Reb': 17,
    'team_Ast': 11,
    'team_Stl': 17,
    'team_Blk': 15,
    'team_TO': 15,
    'team_PF': 20,
}

In [81]:
#generate the X Y matrices
def df_to_X_y(df, dfWindow, dfHomeWindow, dfAwayWindow, window_size):
    X = []
    y = []
    #each team must have played window_size games before data can be extracted
    for index, row in df.iterrows():
        homeTeam_id = row['homeTeamSubject_id']
        awayTeam_id = row['awayTeamSubject_id']
        date = row['date']
        #limit dfWindow to include rows that occured before date of row we are on
        dfWindow_before_date = dfWindow[dfWindow['date']< date]
        dfHomeWindow_before_date = dfHomeWindow[dfHomeWindow['date']< date]
        dfAwayWindow_before_date = dfAwayWindow[dfAwayWindow['date']< date]
        homeTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].shape[0]
        awayTeam_occurrences = dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].shape[0]
        if homeTeam_occurrences > window_size and awayTeam_occurrences > window_size:
            homeTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            awayTeam_window = (dfWindow_before_date[dfWindow_before_date['team_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'team_id']))
            homeTeam_home_window = (dfHomeWindow_before_date[dfHomeWindow_before_date['homeTeamSubject_id'] == homeTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'homeTeamSubject_id']))
            awayTeam_away_window = (dfAwayWindow_before_date[dfAwayWindow_before_date['awayTeamSubject_id'] == awayTeam_id].sort_values(by='date', ascending=False).head(window_size).drop(columns=['date', 'awayTeamSubject_id']))
            #condtional for average or top entry - home Team
            homeTeam_np = []
            awayTeam_np = []
            homeTeam_home_np = []
            awayTeam_away_np = []
            for column in homeTeam_window:
                specified_window = windowConditions.get(column)
                homeTeam_np.append(homeTeam_window[column].iloc[:specified_window].mean(axis=0))
                awayTeam_np.append(awayTeam_window[column].iloc[:specified_window].mean(axis=0))
                homeTeam_home_np.append(homeTeam_home_window[f'homeT{column[1:]}'].iloc[:specified_window].mean(axis=0))
                awayTeam_away_np.append(awayTeam_away_window[f'awayT{column[1:]}'].iloc[:specified_window].mean(axis=0))
            combined_window = np.hstack((np.array(homeTeam_np), np.array(awayTeam_np), np.array(homeTeam_home_np), np.array(awayTeam_away_np)))
            X.append(combined_window)
            y.append(row['winner_binary'])
    
    return np.array(X), np.array(y)

In [82]:
#X1.Shape is num data, window size, variables
X, y = df_to_X_y(df, dfIDtoStat, dfHome, dfAway, 20)
X.shape, y.shape

((25406, 80), (25406,))

In [83]:
#80-10-10 split
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)
X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape

((20324, 80), (20324,), (2541, 80), (2541,), (2541, 80), (2541,))

In [84]:
model = Sequential()
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [85]:
#early stopping if model does not improve performance for 50 epochs
earlystopping = EarlyStopping(monitor='val_loss', patience=50, restore_best_weights=True)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [86]:
#simple model fit
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=100, callbacks=[earlystopping])

Epoch 1/100
636/636 [==============================] - 2s 1ms/step - loss: 0.6431 - accuracy: 0.6311 - val_loss: 0.6666 - val_accuracy: 0.6273
Epoch 2/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6329 - accuracy: 0.6395 - val_loss: 0.6271 - val_accuracy: 0.6411
Epoch 3/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6278 - accuracy: 0.6456 - val_loss: 0.6251 - val_accuracy: 0.6454
Epoch 4/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6289 - accuracy: 0.6464 - val_loss: 0.6268 - val_accuracy: 0.6466
Epoch 5/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6263 - accuracy: 0.6464 - val_loss: 0.6222 - val_accuracy: 0.6482
Epoch 6/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6251 - accuracy: 0.6492 - val_loss: 0.6233 - val_accuracy: 0.6529
Epoch 7/100
636/636 [==============================] - 1s 1ms/step - loss: 0.6256 - accuracy: 0.6498 - val_loss: 0.6195 - val_accuracy: 0.6525

In [87]:
y_pred = model.predict(X_test)
y_pred_binary = (y_pred > 0.5).astype(int)
print(accuracy_score(y_test, y_pred_binary))

80/80 [==============================] - 0s 671us/step
0.6619441164895711
